In [1]:
%matplotlib widget
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as anim

from src.env.Environment import BoxEnv, Box
from src.env.Transform import Transform
from src.env.Skeleton import Skeleton
from src.env.Environment import Interactive
from src.data.ShapeManager import Data
from src.data.SkeletonFrame import SkeletonFrame

In [3]:
X = Data(np.load('./data/test16.npy'), np.load('./data/input_norm.npy'), "input")


data = np.load('data/test16.npy').astype(np.float32)
sequences = np.loadtxt('./data/TestSequences.txt')

input_norm = np.load('data/input_norm.npy').astype(np.float32)
output_norm = np.load('data/output_norm.npy').astype(np.float32)

input_data = Data(data[:,:input_norm.shape[1]], input_norm, "input", sequences)
output_data = Data(data[:,input_norm.shape[1]:], output_norm, "output", sequences)

In [4]:
class WorldFrame:
    def __init__(self, data):       
        self.data = data
        self.projection = "3d"

        self.bone_order = list(range(7)) + [5] + list(range(7,11)) \
                + list(range(10,6,-1)) + [5] + list(range(11,15)) \
                + list(range(14,10,-1)) + list(range(5,-1,-1)) \
                    + list(range(15,19)) + list(range(18,14,-1)) + [0] \
                        + list(range(19,23))

    def draw_points(self, ax, points, size = 5, alpha = 1., linestyle = "", linewidth = 3, marker = "o", c = None):
        return ax.plot(points[:,0],points[:,2],points[:,1], linestyle=linestyle, c = c, linewidth=linewidth, marker=marker, markersize = size, alpha = alpha)[0]
    def update_points(self, graph, points, size = 5):
        graph.set_data(points[:,0],points[:,2])
        graph.set_3d_properties(points[:,1])

    def draw_goals(self, ax):
        return self.draw_points(ax, self.data.goal_position, linestyle=None)
    def update_goals(self, graph):
        self.update_points(graph, self.data.goal_position)

    def draw_trajectory(self, ax):
        return ax.plot(
            self.data.trajectory_position[:,0],
            self.data.trajectory_position[:,1],
            self.data.trajectory_position[:,0] * 0,
            marker="o"
        )[0]
    def update_trajectory(self, graph):
        graph.set_data(
            self.data.trajectory_position[:,0],
            self.data.trajectory_position[:,1]
        )
        graph.set_3d_properties(
            self.data.trajectory_position[:,0] * 0,
        )

    def draw_interaction(self, ax):
        return self.draw_points(
            ax, 
            self.data.interaction_position[self.data.interaction_occupancy > 1e-5]
        )
    def update_interaction(self, graph):
        self.update_points(
            graph, 
            self.data.interaction_position[self.data.interaction_occupancy > 1e-5]
        )

    def draw_environment(self, ax):
        points = gen_points(4, 9, 9)
        points = points[self.data.environment > 0]
        return self.draw_points(
            ax, 
            points, 
            alpha = 0.5, 
            size = 15
        )
    def update_environment(self, graph):
        points = gen_points(4, 9, 9)
        points = points[self.data.environment > 1e-5]
        self.update_points(graph, points)

    def draw_character(self, ax):
        return self.draw_points(ax, self.data.bone_position[self.bone_order], linestyle="-")

    def update_character(self, graph):
        self.update_points(graph, self.data.bone_position[self.bone_order])

    def draw(self, ax):
        ax.set_yticklabels([])
        ax.set_xticklabels([])
        ax.set_zticklabels([])

        graph = [
            self.draw_goals(ax),
            self.draw_trajectory(ax),
            self.draw_environment(ax),
            self.draw_interaction(ax),
            self.draw_character(ax)
        ]

        return graph

    def update(self, graph):
        func = [
            self.update_goals,
            self.update_trajectory,
            self.update_environment,
            self.update_interaction,
            self.update_character
        ]
        for f,x in zip(func, graph):
            f(x)
        return graph

In [5]:
class World:
    def __init__(self, env, objects):
        self.skeleton = Skeleton([0,0],[1,0])
        self.env = env
        self.objects = objects
        self.obj_idx = -1
        
        self.init()
    
    def init(self):
        fig = plt.figure()
        self.ax = fig.add_subplot(111,projection='3d')

        self.ax.set_xlim3d(-3,3)
        self.ax.set_ylim3d(-3,3)
        self.ax.set_zlim3d(0,6)
        
        self.frame = SkeletonFrame(self.skeleton.current)        
        self.graph = self.frame.draw(self.ax)
        
    def get_data(self):
        data = self.skeleton.current
        collision = self.env.collide(self.skeleton)                
    
    def draw(self):
        pass
    
    def update(self):
        self.skeleton.update(self.env, [])
        self.frame = SkeletonFrame(self.skeleton.current)
        self.frame.update(self.graph)

        
        return self.graph
        
    def play(self):
        self.anim = anim.FuncAnimation(self.fig, self.update, interval = 1000 / 120., blit=True)

    def pause(self):
        self.is_paused = True

    def resume(self):
        self.is_paused = False


In [6]:
env = BoxEnv([
    Box()
])
obj_list = []
world = World(env, obj_list)

/home/hayk/.pyenv/versions/ai/lib/python3.7/site-packages/torch/serialization.py:657: SourceChangeWarning: source code of class 'src.nn.nets.NSM' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
current = data[0]
next = data[1]

In [8]:
input_data[0].trajectory_position[8]

array([0.00174046, 0.00301933], dtype=float32)

In [46]:
np.linalg.norm(input_data[0].trajectory_direction, axis = 1)

array([1.0000751, 1.0001705, 1.0000502, 1.0000007, 1.000051 , 1.000018 ,
       1.       , 1.0000002, 1.       , 1.0000069, 1.0000104, 1.0000201,
       1.000011 ], dtype=float32)

In [52]:
np.cos(angle)

6.123233995736766e-17

In [53]:
np.sin(angle)

1.0

In [10]:
traj = input_data.trajectory_position

In [14]:
A = np.insert(traj, 1, 0, axis = 2)

In [27]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure()
ax = fig.add_subplot(111,projection='3d')
ax.set_xlim3d(-3,3)

xx, yy = np.mgrid[-3:3,-3:3]
z = 0 * xx

ax.plot_surface(xx, yy, z, alpha=0.2)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

array([[[-3, -3, -3, -3, -3, -3],
        [-2, -2, -2, -2, -2, -2],
        [-1, -1, -1, -1, -1, -1],
        [ 0,  0,  0,  0,  0,  0],
        [ 1,  1,  1,  1,  1,  1],
        [ 2,  2,  2,  2,  2,  2]],

       [[-3, -2, -1,  0,  1,  2],
        [-3, -2, -1,  0,  1,  2],
        [-3, -2, -1,  0,  1,  2],
        [-3, -2, -1,  0,  1,  2],
        [-3, -2, -1,  0,  1,  2],
        [-3, -2, -1,  0,  1,  2]]])

In [15]:
np.delete(A, 1, axis = 2)

array([[[ 9.3612671e-03, -4.0397644e-03],
        [ 9.1476440e-03, -7.0495605e-03],
        [ 7.2593689e-03, -8.8272095e-03],
        ...,
        [ 1.7995834e-03,  9.6588135e-03],
        [-6.0021877e-05,  1.6464233e-02],
        [-5.3787231e-03,  2.7099609e-02]],

       [[ 9.3688965e-03, -5.1116943e-03],
        [ 8.7966919e-03, -7.9498291e-03],
        [ 6.6719055e-03, -9.3078613e-03],
        ...,
        [ 1.5401840e-03,  1.0360718e-02],
        [-8.4018707e-04,  1.7776489e-02],
        [-7.1601868e-03,  2.9541016e-02]],

       [[ 9.2773438e-03, -6.0997009e-03],
        [ 8.3084106e-03, -8.8195801e-03],
        [ 5.9814453e-03, -9.5291138e-03],
        ...,
        [ 1.1100769e-03,  1.1222839e-02],
        [-1.8701553e-03,  1.9226074e-02],
        [-9.1629028e-03,  3.2196045e-02]],

       ...,

       [[ 1.2580872e-02, -1.1220703e+00],
        [ 5.1513672e-02, -9.4873047e-01],
        [ 7.7026367e-02, -8.1640625e-01],
        ...,
        [ 7.4218750e-02,  8.6279297e-01],
     

In [94]:
b = Box(Transform.from_rot_pos(
    [45,0,0],
    [1,1,1]
))
drawing.draw_cube(b.get_def())

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …